# 导入数据

### 解压文件

In [ ]:
import tarfile
tar = tarfile.open('/content/CUB_200_2011.tgz')     
tar.extractall('/content')                      # 解压文件到/content目录下
tar.close()

### 读取数据并划分训练集、测试集和验证集

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split

def get_data_loaders(data_dir, batch_size):
  """
  将图像重构成224*224的像素
  shuffle=True时，数据加载器会在每个时期重新洗牌数据，以增加训练的随机性。这对于避免模型过度拟合训练数据、增加模型的泛化能力非常重要。
  DataLoader类创建训练集、验证集和测试集的数据加载器。数据加载器用于批量加载数据，并提供数据的迭代器。
  将同一类的图像放在一个文件夹中，此函数会自动分辨哪个图像属于哪一类.
  接受两个参数：data_dir（数据集所在的目录）和batch_size（批量大小）
  """
  transform = transforms.Compose([transforms.Resize(256),
                   transforms.CenterCrop(224),
                   transforms.ToTensor()])
  all_data = datasets.ImageFolder(data_dir, transform=transform)
  train_data_len = int(len(all_data)*0.75)
  valid_data_len = int((len(all_data) - train_data_len)/2)
  test_data_len = int(len(all_data) - train_data_len - valid_data_len)
  train_data, val_data, test_data = random_split(all_data, [train_data_len, valid_data_len, test_data_len])
  train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
  #使用
  #
  return ((train_loader, val_loader, test_loader),train_data, val_data, test_data, all_data.classes)

(train_loader, val_loader, test_loader),train_data, val_data, test_data, classes = get_data_loaders("images_path/", 64)

### 将图像进行归一化处理

In [ ]:
def normalize_image(image):
    """ 
    通过调用 clamp_ 方法，将图像张量中的值约束在 image_min 和 image_max 之间
    调用 add_ 方法将 image_min 从图像张量中减去，使得最小值变为 0
    调用 div_ 方法将图像张量除以 image_max - image_min + 1e-5，进行归一化操作
    最后，返回归一化后的图像张量

    """
    image_min = image.min()
    image_max = image.max()
    image.clamp_(min = image_min, max = image_max)
    image.add_(-image_min).div_(image_max - image_min + 1e-5)
    return image

### 展示图像和标签

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def plot_images(images, labels, normalize = True):
    """ 
    plot_images 的函数，用于绘制图像和对应的标签
    引用了 normalize_image 函数，因此在调用 plot_images 函数之前，需要确保已经定义了 normalize_image 函数。

    """
    n_images = len(images)
    rows = int(np.sqrt(n_images))
    cols = int(np.sqrt(n_images))
    fig = plt.figure(figsize = (15, 15))
    for i in range(rows*cols):
        ax = fig.add_subplot(rows, cols, i+1)
        image = images[i]
        if normalize:
            image = normalize_image(image)
        ax.imshow(image.permute(1, 2, 0).cpu().numpy())
        label = labels[i]
        ax.set_title(label)
        ax.axis('off')


N_IMAGES = 25    #从训练数据中选择前 N_IMAGES 个样本，并将它们的图像和标签传递给 plot_images 函数进行绘制
images, labels = zip(*[(image, label) for image, label in
                           [train_data[i] for i in range(N_IMAGES)]])
plot_images(images, labels)